In [1]:
%pip install requests beautifulsoup4 pandas


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 1. Liste de tes discours (C'est ici que tu colles les URLs que tu as trouvées)
# Remplis cette liste avec les 14 liens.
urls_to_scrape = [
    {"year": 2024, "party": "Democrat", "candidate": "Harris", "url": "https://www.presidency.ucsb.edu/documents/address-accepting-the-democratic-presidential-nomination-chicago-illinois"},
    {"year": 2024, "party": "Republican", "candidate": "Trump", "url": "https://www.presidency.ucsb.edu/documents/address-accepting-the-presidential-nomination-the-republican-national-convention-milwaukee"},
     {"year": 2020, "party": "Democrat", "candidate": "Biden", "url": "https://www.presidency.ucsb.edu/documents/address-accepting-the-democratic-presidential-nomination-wilmington-delaware"},
      {"year": 2020, "party": "Republican", "candidate": "Trump", "url": "https://www.presidency.ucsb.edu/documents/address-accepting-the-republican-presidential-nomination-2"},
      {"year": 2016, "party": "Democrat", "candidate": "Clinton", "url": "https://www.presidency.ucsb.edu/documents/address-accepting-the-presidential-nomination-the-democratic-national-convention"},
      {"year": 2016, "party": "Republican", "candidate": "Trump", "url": "https://www.presidency.ucsb.edu/documents/address-accepting-the-presidential-nomination-the-republican-national-convention-cleveland"},
      {"year": 2012, "party": "Democrat", "candidate": "Obama", "url": "https://www.presidency.ucsb.edu/documents/remarks-accepting-the-presidential-nomination-the-democratic-national-convention-charlotte"},
      {"year": 2012, "party": "Republican", "candidate": "Romney", "url": "https://www.presidency.ucsb.edu/documents/address-accepting-the-presidential-nomination-the-republican-national-convention-tampa"},
      {"year": 2008, "party": "Democrat", "candidate": "Obama", "url": "https://www.presidency.ucsb.edu/documents/address-accepting-the-presidential-nomination-the-democratic-national-convention-denver"},
      {"year": 2008, "party": "Republican", "candidate": "McCaine", "url": "https://www.presidency.ucsb.edu/documents/address-accepting-the-presidential-nomination-the-republican-national-convention-saint"},
      {"year": 2004, "party": "Democrat", "candidate": "Kerry", "url": "https://www.presidency.ucsb.edu/documents/address-accepting-the-presidential-nomination-the-democratic-national-convention-boston"},
      {"year": 2004, "party": "Republican", "candidate": "Bush", "url": "https://www.presidency.ucsb.edu/documents/remarks-accepting-the-presidential-nomination-the-republican-national-convention-new-york"},
      {"year": 2000, "party": "Democrat", "candidate": "Gore", "url": "https://www.presidency.ucsb.edu/documents/address-accepting-the-presidential-nomination-the-democratic-national-convention-los"},
      {"year": 2000, "party": "Republican", "candidate": "Bush", "url": "https://www.presidency.ucsb.edu/documents/address-accepting-the-presidential-nomination-the-republican-national-convention-0"},
   

    # ... ajoute les autres années ici (2020, 2016, etc.) ...
]

speech_data = []

# 2. La boucle de scraping
for item in urls_to_scrape:
    print(f"Récupération de : {item['candidate']} ({item['year']})...")
    
    try:
        # On va chercher la page
        response = requests.get(item['url'])
        
        # On vérifie que la page répond bien (Code 200)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # --- C'est ici la partie "Magique" ---
            # Sur "The American Presidency Project", le texte est souvent dans <div class="field-docs-content">
            # On cherche cette section spécifique
            content_div = soup.find('div', class_='field-docs-content')
            
            if content_div:
                # On récupère tout le texte sans les balises HTML
                text = content_div.get_text(separator=' ', strip=True)
                
                # On sauvegarde dans notre liste
                speech_data.append({
                    "year": item['year'],
                    "party": item['party'],
                    "candidate": item['candidate'],
                    "text": text
                })
            else:
                print(f"ERREUR: Pas de texte trouvé pour {item['candidate']}")
        else:
            print(f"ERREUR: Lien mort pour {item['candidate']}")
            
        # Petite pause pour être poli avec le serveur
        time.sleep(1)
        
    except Exception as e:
        print(f"Erreur critique : {e}")

# 3. Création du DataFrame final
df_speeches = pd.DataFrame(speech_data)

# Afficher les premières lignes pour vérifier
print("Scraping terminé !")
df_speeches.head()

Récupération de : Harris (2024)...
Récupération de : Trump (2024)...
Récupération de : Biden (2020)...
Récupération de : Trump (2020)...
Récupération de : Clinton (2016)...
Récupération de : Trump (2016)...
Récupération de : Obama (2012)...
Récupération de : Romney (2012)...
Récupération de : Obama (2008)...
Récupération de : McCaine (2008)...
Récupération de : Kerry (2004)...
Récupération de : Bush (2004)...
Récupération de : Gore (2000)...
Récupération de : Bush (2000)...
Scraping terminé !


,year,party,candidate,text
0,2024,Democrat,Harris,The Vice President: Good evening! [ Laughs. ] ...
1,2024,Republican,Trump,"Thank you very much. Thank you very, very much..."
2,2020,Democrat,Biden,"Good evening. Ella Baker, a giant of the civil..."
3,2020,Republican,Trump,Thank you very much. Thank you very much. Than...
4,2016,Democrat,Clinton,"Thank you all very, very much! Thank you for t..."


In [5]:
import re

def super_cleaning(text):
    # 1. Enlever tout le contenu entre crochets [ ... ] (Rires, Applaudissements)
    text = re.sub(r'\[.*?\]', '', text)
    
    # 2. Enlever tout le contenu entre parenthèses ( ... )
    text = re.sub(r'\(.*?\)', '', text)
    
    # 3. Enlever les préfixes de type "The Vice President:" au début (si présents)
    if ":" in text[:50]: # Regarde s'il y a deux points au début
        text = text.split(":", 1)[1]
    
    # 4. Nettoyer les espaces multiples et sauts de ligne
    text = text.replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Appliquer le nettoyage
df_speeches['clean_text'] = df_speeches['text'].apply(super_cleaning)

# Comparer Avant / Après sur le premier discours
print("BRUT  :", df_speeches['text'].iloc[0][:150])
print("CLEAN :", df_speeches['clean_text'].iloc[0][:150])

# Sauvegarder la version PROPRE
df_speeches.to_csv("president_speeches_clean.csv", index=False)
print("\n✅ Fichier 'president_speeches_clean.csv' sauvegardé !")

BRUT  : The Vice President: Good evening! [ Laughs. ] [ Applause. ] Audience: Kamala! Kamala! Kamala! The Vice President: California. [ Laughs. ] [ Applause. 
CLEAN : Good evening! Audience: Kamala! Kamala! Kamala! The Vice President: California. Good evening, everyone. Good evening. Good evening. Oh, my goodness. G

✅ Fichier 'president_speeches_clean.csv' sauvegardé !


In [7]:
import pandas as pd
import re

# 1. Charger le fichier actuel
df = pd.read_csv('president_speeches_clean.csv')

# 2. Fonction de nettoyage ULTIME
def ultimate_cleaning(text):
    if not isinstance(text, str): return ""
    
    # Enlever le contenu entre crochets [] et parenthèses ()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\(.*?\)', '', text)
    
    # Enlever les indicateurs de locuteurs fréquents (ex: "Audience:", "The Vice President:")
    # On enlève tout ce qui ressemble à "Mot: " ou "Mot Mot: " en début de phrase ou après ponctuation
    text = re.sub(r'\b(Audience|The Vice President|The President|Hon\.|Mr\.|Ms\.|Mrs\.)\s*:', '', text, flags=re.IGNORECASE)
    
    # Nettoyage standard (espaces)
    text = text.replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# 3. Appliquer le nettoyage
print("Nettoyage en cours...")
df['clean_text'] = df['text'].apply(ultimate_cleaning)

# 4. SÉLECTIONNER UNIQUEMENT LA COLONNE PROPRE
# On ne garde que l'année, le parti, le candidat et le texte propre
df_final = df[['year', 'party', 'candidate', 'clean_text']].copy()

# On renomme 'clean_text' en 'text' pour que ce soit le standard
df_final.rename(columns={'clean_text': 'text'}, inplace=True)

# 5. Sauvegarder en écrasant l'ancien fichier
df_final.to_csv("president_speeches_clean.csv", index=False)

print("✅ Fichier 'president_speeches_clean.csv' mis à jour !")
print("Il ne contient plus que la version propre.")
print("\nAperçu (50 premiers caractères) :")
print(df_final['text'].iloc[0][:100])

Nettoyage en cours...
✅ Fichier 'president_speeches_clean.csv' mis à jour !
Il ne contient plus que la version propre.

Aperçu (50 premiers caractères) :
Good evening! Kamala! Kamala! Kamala! California. Good evening, everyone. Good evening. Good evening
